<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

# Instalacja pakietów

Instalacja kluczowych pakietów
*   requests - Pakiet Python umożliwiający wysyłanie żądań HTTP w prosty sposób. Umożliwia wykonywanie wszystkich typów żądań HTTP, takich jak GET, POST, PUT, DELETE i inne.
*   accelerate - Pomaga w przyspieszaniu i zarządzaniu treningu oraz inferencją modeli na różnych platformach jak GPU, TPU czy CPU.
*   bitsandbytes - Biblioteka używana do optymalizacji wydajności obliczeń, poprzez oferowanie niskopoziomowych operacji arytmetycznych na danych numerycznych.
*   chromadb - Wektorowa baza danych służąca do przechowywania i wyszukiwania wektorów. Przydatna do różnych zastosowań związanych z uczeniem maszynowym i przetwarzaniem języka naturalnego.
*   sentence-transformers - Pakiet Python do generowania osadzeń zdań (embeddings) za pomocą różnych modeli transformerowych. Jest używany w zadaniach takich jak wyszukiwanie semantyczne, klasteryzacja, klasyfikacja i inne.
*   PyMuPDF (fitz) - Biblioteka Python umożliwiająca pracę z plikami PDF i innymi formatami dokumentów. Umożliwia odczyt, zapis i manipulację plikami PDF.










In [ ]:
!pip install requests
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install chromadb
!pip install sentence-transformers
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple/


In [ ]:
# moduł warnings wykluczy nadmiarowe informacje, które mogą wystąpić podczas inferencji modelu
import warnings
warnings.filterwarnings("ignore")

# Deklaracja ścieżek i nazw plików

In [ ]:
pdf_url = "https://www.iab.org.pl/wp-content/uploads/2024/04/Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.pdf"
file_name = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.pdf"
file_json = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.json"
data_dir = "./data"

# Pobieranie pliku PDF

In [ ]:
import requests

response = requests.get(pdf_url, stream=True)
with open(file_name, "wb") as file:
    for block in response.iter_content(chunk_size=1024):
        if block:
            file.write(block)

# Ekstrakcja tekstu z pliku PDF

In [ ]:
import fitz
import json

document = fitz.open(file_name)
text = ''
pages = []

for page_num in range(len(document)):
    page = document.load_page(page_num)
    page_text = page.get_text()
    pages.append({"page_num": page_num, "text": page.get_text()})

with open(file_json, "w") as file:
    json.dump(pages, file, indent=4, ensure_ascii=False)

# Stworzenie osadzeń tekstów
Przy zastosowaniu modelu sentence transformer (model do embeddingów) w postaci silver-retrivera-base-v.1.1
https://huggingface.co/ipipan/silver-retriever-base-v1

In [ ]:
from sentence_transformers import SentenceTransformer

texts = []
for page in pages:
    texts.append(page["text"])

embedding_model = SentenceTransformer('ipipan/silver-retriever-base-v1.1')
embeddings = embedding_model.encode(texts)

# Tworzenie bazy wektorów
W celu przechowywania i wyszukiwania dokumentów na bazie ich osadzeń (embeddingów)

In [ ]:
import chromadb

client = chromadb.PersistentClient(path=data_dir)
collection = client.get_or_create_collection(
        name="ha_naive_rag", metadata={"hnsw:space": "cosine"}
)

collection.add(
        documents=texts,
        embeddings=embeddings,
        ids=[str(i) for i in range(len(texts))]
)

# Gated models
W przypadku modeli, na które zostały nałożone zabezpieczenia w postaci kontrolowanego dostępu (gated models), wymagane będzie podanie tokena użytkownika, dostępnego na portalu HuggingFace: <br>
- informacje dotyczące tokenów: <br>
https://huggingface.co/docs/hub/security-tokens <br>
- token użytkownika <br>
https://huggingface.co/settings/tokens

W tym celu należy zainstalować bibliotekę "huggingface_hub[cli]" oraz dokonać logowania za pomocą tokena, co zostało zaprezentowane w poniższych 2 komórkach.

In [ ]:
!pip install -U "huggingface_hub[cli]"

In [ ]:
!huggingface-cli login

#Zapytajmy chatbota

Jest to prosty bot w stylu **question answering** (**Q&A**) wykorzystujący technikę **RAG** w wersji **tzw. naiwnej**. Pamiętajmy o lepszym chunkowaniu, dokładniejszej analizie struktury dokumentów, wzbogaceniu tekstu, augmentacji zapytań, hybrydowym wyszukiwaniu, dodaniu kontekstu i historii rozmowy, a o**trzymamy PRAWDZIWEGO CHATBOTA w wersji enterprise.** Jednak powyższy kod to kilkanaście linijek kodu, a chatbot przewyższa kilkakrotnie klasyczne chatboty oparte na rozpoznawaniu intencji i zaawansowanych scenariuszach.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer

device = "cuda"
model_name = 'speakleash/Bielik-11B-v2.2-Instruct'

temperature = 1.0
max_tokens = 500
top_k = 200
top_p = 1

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             quantization_config=quantization_config
                                             )

tokenizer_config.json:   0%|          | 0.00/27.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/2.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/151 [00:00<?, ?B/s]

In [ ]:
import uuid

In [ ]:
def create_context(query, top=3):
    embedding = embedding_model.encode([query])

    result = collection.query(
            query_embeddings=embedding,
            n_results=top
    )

    if result:

        ids = result.get("ids", [])
        documents = result.get("documents", [])

        if documents:
            context = "KONTEKST:\n"
            context = context + documents[0][0] + "\n\n"
            if len(documents) > 1:
                context = context + documents[0][1] + "\n\n"
        else:
            print("Brak dokumentów w wynikach.")
            return None

        # zapis do pliku
        filename = f"{str(uuid.uuid4())[:8]}_context.json"

        with open(filename, "w") as file:
            content = {
                    "context": context,
                    "query": query
            }
            json.dump(content, file, ensure_ascii=False, indent=4)

        return f"Wyłącznie na podstawie podanego kontekstu odpowiedz zwięźle na pytanie: '{query}'\n{context}"

    else:
        print("Brak kontekstu")
        return None

In [ ]:
def generate(prompt, system=None):
    messages = []

    if system:
        messages.append({"role": "system", "content": system})

    messages.append({"role": "user", "content": prompt})

    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = input_ids

    if torch.cuda.is_available():
        model_inputs = input_ids.to(device)

    # generowanie odpowiedzi
    outputs = model.generate(model_inputs,
                             streamer=streamer,
                             max_new_tokens=max_tokens,
                             do_sample=True if temperature else False,
                             temperature=temperature,
                             top_k=top_k,
                             top_p=top_p)

    # zapis do pliku
    filename = f"{str(uuid.uuid4())[:8]}.json"

    with open(filename, "w") as file:
        content = {
                "prompt": messages,
                "output": tokenizer.batch_decode(outputs, skip_special_tokens=False)
        }
        json.dump(content, file, ensure_ascii=False, indent=4)


In [ ]:
temperature = 0.1
max_tokens = 1000
top_k = 200
top_p = 1

prompt = create_context(query = "Czy AI zniszczy ludzkość?", top = 3)
generate(prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Na podstawie podanego kontekstu można odpowiedzieć na pytanie "Czy AI zniszczy ludzkość?" w następujący sposób:

"Nie ma jednoznacznej odpowiedzi na to pytanie. Z jednej strony, istnieją obawy, że niewłaściwe zarządzanie AI może prowadzić do scenariuszy, w których maszyny mogłyby zagrozić ludzkości (np. wizja Stephena Hawkinga). Z drugiej strony, eksperci, tacy jak Bill Gates, podkreślają możliwość stworzenia regulacji i nadzoru, które pozwolą na bezpieczny rozwój AI. Fei-Fei Li z kolei wskazuje na potencjał współpracy między ludźmi a AI, gdzie maszyny rozszerzają nasze możliwości, a nie zastępują nas.

Według artykułu z Harvard Business School, ludzie mogą współpracować z AI na dwa sposoby: jako centaury (dzieląc zadania) lub jako cyborgi (łącząc wysiłki). W przyszłości możemy spodziewać się powstania super zespołów, łączących człowieka, robota kolaboratywnego i robota programowego.

Podsumowując, AI ma potencjał zarówno do zagrożenia, jak i wspierania ludzkości, a kluczowe jest odpo